In [1]:
# GET PARAMETERS
import os 
import sys
import torch 
# Get Parent folder : 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern  
from examples.benchmark import local_get_args
# Init:
dataset_names = ['subway_in','subway_indiv'] # ['subway_in'] #["subway_in","subway_out"] # ['subway_in','netmob_POIs_per_station'],["subway_in","subway_out"],["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'ASTGCN' # 'STGCN', 'ASTGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'target_data': 'subway_in', # 'subway_in'
                'freq': '15min',
                'use_target_as_context': False,
                
                
                'epochs' : 1, #100
                'lr':5e-5,# 4e-4,
                'weight_decay':0.05,
                'dropout':0.15,
                #'set_spatial_units':  station,   
                #  
                'scheduler':None,
                'adj_type':'corr',
                'threshold': 0.7,
                #'scheduler':True,
                #'torch_scheduler_milestone': 5,
                #'torch_scheduler_gamma':0.997,
                #'torch_scheduler_lr_start_factor':1,

                'temporal_graph_transformer_encoder': False,
                #'TGE_num_layers' : 4, #2
                #'TGE_num_heads' :  1, #IMPOSSIBLE > 1 CAR DOIT DIVISER L = 7
                #'TGE_FC_hdim' :  32, #32

                #'NetMob_only_epsilon': True,    # True # False
                #'NetMob_selected_apps': ['Apple_iMessage','Web_Ads'],# ['Apple_iMessage','Web_Ads'], #,'Deezer','WhatsApp','Twitter'] #['Google_Maps']# ['Instagram','Google_Maps','Twitter']
                #'NetMob_transfer_mode' :  ['DL'], #,'UL'] # ['DL'] # ['UL'] #['DL','UL']
                #'NetMob_selected_tags' : ['station_epsilon100'],#['iris','stadium','station','university']#['park','stadium','university','station','shop','nightclub','parkings','theatre','iris','transit','public_transport']
                #'NetMob_expanded' : '', # '' # '_expanded'
                'stacked_contextual': True, # True # False

                ### EXIST ONLY IF MODEL = STGCN
                #'learnable_adj_matrix' : False, # True
                #'graph_conv_type': 'graph_conv', # 'cheb_graph_conv', 'graph_conv'
                #'learnable_adj_top_k': 10,
                #'learnable_adj_embd_dim': 16, 
                ### ========

                #'vision_num_heads':6,
                #"vision_grn_out_dim":48,
                #'vision_model_name': 'VariableSelectionNetwork',
                #'vision_concatenation_early':True,   
                #'vision_concatenation_late':True,
                'compute_node_attr_with_attn' : False, # True ??
                #'adj_type':'corr'
                           }
# ...
#1038945

# Training and visu: 
args = local_get_args(model_name,
                    args_init = None,
                    dataset_names=dataset_names,
                    dataset_for_coverage=dataset_for_coverage,
                    modification = modification)

training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init = args,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

Training and Hyper-parameter tuning with Ray is not possible
>>>>Model: ASTGCN; K_fold = 6; Loss function: MSE 
>>>>Model: ASTGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
>>>Tackle dataset_name: subway_indiv
Chargement des données depuis : /home/rrochas/prediction-validation/../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle/subway_indiv_15min/subway_indiv_15min.csv
df.head():    COD_TRG             VAL_DATE  Flow
0     AMP  2020-02-01 00:00:00    49
1     AMP  2020-02-01 00:15:00    14
2     AMP  2020-02-01 00:30:00    43
3     AMP  2020-02-01 00:45:00    17
4     AMP  2020-02-01 01:00:00    20
Len coverage_period:  7392
df.head():    COD_TRG            VAL_DATE  Flow
0     AMP 2020-02-01 00:00:00    49
1     AMP 2020-02-01 00:15:00    14
2     AMP 2020-02-01 00:30:00    43
3     AMP 2020-02-01 00:45:00    17
4     AMP 2020-02-01 01:00:00   

AttributeError: 'NoneType' object has no attribute 'name'

In [9]:
import pandas as pd
dirname = '/home/rrochas/prediction-validation/../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle/subway_indiv_15min/'
os.listdir(dirname)
os.path.exists(f"{dirname}/subway_indiv_15min.csv")

FILE_BASE_NAME = 'subway_indiv'
DATA_SUBFOLDER = 'agg_data/validation_individuelle' # Sous-dossier dans FOLDER_PATH
NATIVE_FREQ = '3min'
START = '2019-10-01' # Exemple basé sur head()
END = '2020-04-01'
list_of_invalid_period = []
C = 1

DATE_COL = 'VAL_DATE'
LOCATION_COL = 'COD_TRG'
VALUE_COL = 'Flow'

target_freq = '15min'
file_name = f"{FILE_BASE_NAME}_{target_freq}"
data_file_path = f"{dirname}/{file_name}.csv"

df = pd.read_csv(data_file_path)
df['VAL_DATE'] = pd.to_datetime(df['VAL_DATE'])
df=df.set_index('VAL_DATE')
df.index.unique().sort_values()

DatetimeIndex(['2020-02-01 00:00:00', '2020-02-01 00:15:00',
               '2020-02-01 00:30:00', '2020-02-01 00:45:00',
               '2020-02-01 01:00:00', '2020-02-01 01:15:00',
               '2020-02-01 01:30:00', '2020-02-01 01:45:00',
               '2020-02-01 02:00:00', '2020-02-01 02:15:00',
               ...
               '2020-02-27 21:30:00', '2020-02-27 21:45:00',
               '2020-02-27 22:00:00', '2020-02-27 22:15:00',
               '2020-02-27 22:30:00', '2020-02-27 22:45:00',
               '2020-02-27 23:00:00', '2020-02-27 23:15:00',
               '2020-02-27 23:30:00', '2020-02-27 23:45:00'],
              dtype='datetime64[ns]', name='VAL_DATE', length=2262, freq=None)

In [ ]:
df

In [3]:
trainer.train_and_valid()


start training

Training Throughput:1063.21 sequences per seconds
>>> Training complete in: 0:04:49.253988
>>> Training performance time: min 0.016025543212890625 avg 0.028232812881469727 seconds (+/- 0.007409689556679389)
>>> Loading performance time: min 0.00011897087097167969 avg 0.021025142456797832 seconds (+/- 0.0533871588248097)
>>> Forward performance time: 0.014497923700226319 seconds (+/- 0.006785826679855182)
>>> Backward performance time: 0.01471411531366729 seconds (+/- 0.0026613273244625673)
>>> Plotting performance time: 2.2859429594260365e-06 seconds (+/- 7.815177432756738e-07)
>>> Saving performance time: 0.6629389921824137 seconds (+/- 0.29361714449532417)
>>> PI-tracking performance time: 5.977237643908016e-06 seconds (+/- 7.975377572890917e-06)
>>> Scheduler-update performance time: 4.9972054946362674e-06 seconds (+/- 9.795780319996475e-06)
>>> Validation time: 0:00:00.356755
Proportion of time consumed for Loading: 41.6%
Proportion of time consumed for Forward: 28

In [11]:
from scipy.sparse.linalg import eigs
import numpy as np 
import pandas as pd
L = (pd.DataFrame(np.random.rand(10,10))*10)


lambda_max = eigs(L.values , k=1, which='LR')[0].real

In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
working_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob
from constants.config import get_args,update_modif
from constants.paths import FOLDER_PATH,FILE_NAME,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter
from trainer import Trainer
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from plotting.plotting_bokeh import plot_bokeh


# Load config
model_name = 'STGCN' #'CNN'
dataset_names = ['subway_in','netmob']
args = get_args(model_name,dataset_names)

# Modification : 
args.K_fold = 5

args.ray = False
args.W = 0  # IMPORTANT AVEC NETMOB

args.epochs = 100
args.loss_function_type = 'MSE' # 'quantile'

# optimization:
args.mixed_precision = True

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob','subway_in'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'FeatureExtractor_ResNetInspired'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
folds = [0] # Here we use the first fold for HP-tuning. 

# In case we need to compute the Sliding K-fold validation:
# folds = np.arange(1,args.K_fold)

K_fold_splitter = KFoldSplitter(args,folds)
K_subway_ds,args = K_fold_splitter.split_k_fold()
subway_ds = K_subway_ds[0]

ImportError: cannot import name 'match_period_coverage_with_netmob' from 'utils.utilities_DL' (/home/rrochas/prediction-validation/utils/utilities_DL.py)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [10]:
import torch

netmob_init = torch.Tensor([7392*4*263*287])
netmob_tensor = torch.Tensor([2934*4*263*287*8])

print('Init tensor: ',netmob_init)
print('Init feature vect: ',netmob_tensor)

print('\nblock1:')
sconv = torch.prod(torch.Tensor([64, 32, 131, 143, 8]))
tconv = torch.prod(torch.Tensor([64, 128, 131, 143, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)

print('\nblock2:')
sconv = torch.prod(torch.Tensor([64, 658, 65, 71, 8]))
tconv = torch.prod(torch.Tensor([64, 512, 65, 71, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)


Init tensor:  tensor([2.2318e+09])
Init feature vect:  tensor([7.0868e+09])

block1:
s_conv:  tensor(3.0692e+08) tconv:  tensor(1.2277e+09)

block2:
s_conv:  tensor(1.5548e+09) tconv:  tensor(1.2098e+09)
